# インタラクティブに関数をフィットする方法

関数のフィッティングは一般にめんどくさいですが、少しでも楽にするために**<span style="color: magenta; ">Fitする対象の点を目視しながら選ぶ</span>**ようにするコードを作成しました。

また、今回のコードは基本的に

- [[SciPy] 9. フィッティング範囲をipywidgetsでインタラクティブに変化させてcurve_fit](https://sabopy.com/py/scipy-9/)

- [[ipywidgets] 17. tabで複数のwidgetsを整理して表示](https://sabopy.com/py/ipywidgets-17/)

の内容をフォローしたものです。

## はじめに
---

pythonおよびjupyterlabのインストールをしておく必要があります。詳しくは

- [JupyterLabのすゝめ](https://qiita.com/kirikei/items/a1639954ce5ccaf7ac3c)

- [図解！Jupyter Labを徹底解説！(インストール・使い方・拡張機能)](https://ai-inter1.com/jupyter-lab/)

などを参照すると良いでしょう。

以下、python3をpip3で構築していると仮定して記述します。

## モジュールのimport
---

事前にpythonの各モジュールをPCにインストールしておく必要があります。

### numpy

```
$ pip3 install numpy
```

### matplotlib

```
$ pip3 install matplotlib
$ jupyter labextension install jupyter-matplotlib
```

### scipy

```
$ pip3 install scipy
```

### ipywidgets

```
$ pip3 install ipywidgets
$ jupyter labextension install @jupyter-widgets/jupyterlab-manager
$ jupyter serverextension enable --py --sys-prefix widgetsnbextension
```

以上の環境が整った上で、モジュールをimportします。

In [7]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import optimize
from matplotlib.gridspec import GridSpec

from ipywidgets import interact,interactive,IntSlider,HBox,VBox,Text, Tab,IntRangeSlider
from IPython.display import display

plt.style.use('default')

## データファイルの読み込み
---
このサンプルでは、
```
軸1 データ点1 データ点2 データ点3 データ点4
```
という構造になっているサンプルファイル[test.dat](https://ness1.sci.ibaraki.ac.jp/usefull/test.dat)を使用します。

In [108]:
data_axis1, data_value1, data_value2, data_value3, data_value4 = np.loadtxt("./test.dat", unpack=True)

## 関数の定義
---

- `func_cn(x, a)`

$$
f(x) = a x^n
$$

を定義しています。

- `plot(x_range1, x_range2, x_range3, x_range4)`

この関数がインタラクティブFitの設定になっています。

`start, end` : データ点の範囲指定

`popt, pcov` : 推定パラメーター値・共分散

`optimize.curve_fit` : 最小二乗法によるフィッティング

`y_cf` : 最適化後の関数

`l` : `y_cf`の描画（後述）

`p` : データ点の描画（後述）

In [102]:
def func_c1(x, a):
    return a*x

def func_c2(x, a):
    return a*x**2

def func_c3(x, a):
    return 1.066986166127336e-07 + a*x**3

def func_c4(x, a):
    return a*x**4

def plot(x_range1, x_range2, x_range3, x_range4):
    start1, end1 = x_range1[0], x_range1[1]
    start2, end2 = x_range2[0], x_range2[1]
    start3, end3 = x_range3[0], x_range3[1]
    start4, end4 = x_range4[0], x_range4[1]
    
    popt1, pcov1 = optimize.curve_fit(func_c1, data_axis1[start1:end1], data_value1[start1:end1], p0=params1) 
    popt2, pcov2 = optimize.curve_fit(func_c2, data_axis1[start2:end2], data_value2[start2:end2], p0=params2) 
    popt3, pcov3 = optimize.curve_fit(func_c3, data_axis1[start3:end3], data_value3[start3:end3], p0=params3) 
    popt4, pcov4 = optimize.curve_fit(func_c4, data_axis1[start4:end4], data_value4[start4:end4], p0=params4) 
    
    y_cf1 = func_c1(data_axis1, *popt1)
    y_cf2 = func_c2(data_axis1, *popt2)
    y_cf3 = func_c3(data_axis1, *popt3)
    y_cf4 = func_c4(data_axis1, *popt4)
    
    l1.set_ydata(y_cf1)
    l2.set_ydata(y_cf2)
    l3.set_ydata(y_cf3)
    l4.set_ydata(y_cf4)
    
    p1.set_xdata(data_axis1[start1:end1])
    p2.set_xdata(data_axis1[start2:end2])
    p3.set_xdata(data_axis1[start3:end3])
    p4.set_xdata(data_axis1[start4:end4])
    
    p1.set_ydata(data_value1[start1:end1])
    p2.set_ydata(data_value2[start2:end2])
    p3.set_ydata(data_value3[start3:end3])
    p4.set_ydata(data_value4[start4:end4])

    ax.set_title("$n=1, y=$" + str(np.round(popt1[0], 3)) + "$x$\n" \
                "$n=2, y=$" + str(np.round(popt2[0], 3)) + "$x^2$\n" \
                "$n=3, y=$" + str(np.round(popt3[0], 3)) + "$x^3$\n" \
                "$n=4, y=$" + str(np.round(popt4[0], 3)) + "$x^4$")

## フィッティング
---

`optimize.curve_fit`で各種データに対して最小二乗法によりFitします。

In [111]:
params1, cov1 = optimize.curve_fit(func_c1, data_axis1, data_value1, p0=[1])
params2, cov2 = optimize.curve_fit(func_c2, data_axis1, data_value2, p0=[1])
params3, cov3 = optimize.curve_fit(func_c3, data_axis1, data_value3, p0=[1])
params4, cov4 = optimize.curve_fit(func_c4, data_axis1, data_value4, p0=[1])

y_cf1 = func_c1(data_axis1, *params1)
y_cf2 = func_c2(data_axis1, *params2)
y_cf3 = func_c3(data_axis1, *params3)
y_cf4 = func_c4(data_axis1, *params4)

## プロット＆実行
---

In [114]:
##### プロットエリアの設定 #####
fig = plt.figure(figsize=(8, 8))
gs = GridSpec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[0, 0])
plt.subplots_adjust(top=0.85)
#############################

##### 軸やラベルの設定 #####
ax.grid(True)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(xlabel="$\mathcal{E}_{\mathrm{ac}}/J$", fontsize=16)
ax.set_ylabel(ylabel="$P(n\Omega)$", fontsize=16)
##########################

##### ベースプロットポイントの描画 #####
ax.plot(data_axis1, data_value1, "o", label="$n=1$")
ax.plot(data_axis1, data_value2, "s", label="$n=2$")
ax.plot(data_axis1, data_value3, "^", label="$n=3$")
ax.plot(data_axis1, data_value4, "D", label="$n=4$")
ax.legend()
###################################

##### Fitに使ったデータ点の描画 #####
p1, = ax.plot(data_axis1, data_value1, 'yo', linewidth=1.5)
p2, = ax.plot(data_axis1, data_value2, 'ys', linewidth=1.5)
p3, = ax.plot(data_axis1, data_value3, 'y^', linewidth=1.5)
p4, = ax.plot(data_axis1, data_value4, 'yD', linewidth=1.5)
##################################

##### Fitしたラインの描画 #####
l1, =ax.plot(data_axis1,y_cf1, "-")
l2, =ax.plot(data_axis1,y_cf2, "-")
l3, =ax.plot(data_axis1,y_cf3, "-")
l4, =ax.plot(data_axis1,y_cf4, "-")
#############################

##### スライダーの設定 #####
x_range1 = IntRangeSlider(min=0, max=len(data_axis1), step=1, value=[0,len(data_axis1)], description="x_range1: ", orientation='horizontal')
x_range2 = IntRangeSlider(min=0, max=len(data_axis1), step=1, value=[0,len(data_axis1)], description="x_range2: ", orientation='horizontal')
x_range3 = IntRangeSlider(min=0, max=len(data_axis1), step=1, value=[0,len(data_axis1)], description="x_range3: ", orientation='horizontal')
x_range4 = IntRangeSlider(min=0, max=len(data_axis1), step=1, value=[0,len(data_axis1)], description="x_range4: ", orientation='horizontal')
#########################            

##### インタラクティブ描画の本体部分 #####
interactive(plot, x_range1=x_range1, x_range2=x_range2, x_range3=x_range3, x_range4=x_range4)
#####################################

##### スライダーをどういうふうに並べるかの設定 #####
hbox1 = HBox([x_range1, x_range2])
hbox2 = HBox([x_range3, x_range4])
ui = VBox([hbox1, hbox2])
#############################################

##### スライダーの描画 #####
display(ui)
#########################

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …